'''
【课程3.4】  Python数据预处理

将数据中导演与演员的关系整理出来，得到导演与演员的关系数据，并统计合作次数

'''

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

import warnings
warnings.filterwarnings('ignore') 


In [5]:
import os
os.chdir('D:\\user_profile\\python\\data visualization\\data relationship\\data cleaning')
df = pd.read_excel('豆瓣电影数据.xlsx',sheet_name=0,header=0)
print(df.head(2))
print(f'数据一共有{len(df)}条')
print(f'数据的字段为:\n',df.columns.tolist())

      imdb_id  imdb_n_comment  imdb_score   name  rating_people  score  \
0   tt1587878           222.0         7.0     团圆         4247.0    7.6   
1   tt1810602           999.0         6.0  非诚勿扰2       216051.0    6.5   

                                    上映日期  \
0   2013-09-19(中国大陆) / 2010-02-11(柏林电影节)   
1                       2010-12-22(中国大陆)   

                                                  主演 制片国家/地区  \
0                          卢燕 / 凌峰 / 徐才根 / 马晓晴 / 莫小棋    中国大陆   
1   葛优 / 舒淇 / 孙红雷 / 姚晨 / 安以轩 / 邵兵 / 廖凡 / 邬逸聪 / 关晓...    中国大陆   

                            又名  ...        片长        类型         编剧  \
0               Apart Together  ...      97分钟   剧情 / 家庭   金娜 / 王全安   
1   不省心 / If You Are The One 2  ...     120分钟   喜剧 / 爱情   王朔 / 冯小刚   

             语言 主演人数     r5     r4     r3     r2     r1  
0   上海话 / 汉语普通话    5  0.177  0.497  0.283  0.034  0.008  
1         汉语普通话   15  0.077  0.285  0.486  0.124  0.029  

[2 rows x 21 columns]
数据一共有2876条
数据的字段为:
 ['imdb_id', 'imdb_n_com

In [15]:
# 数据清洗

data = df[['name','导演','主演']]                       # 提取数据
print(data.head(2))
data.dropna(inplace=True)                               # 删除缺失值

data_yy = data['主演'].str.split('/ ',expand=True)      # 演员数据分列
col_len1 = len(data_yy.columns)
#print(col_len1)
data_yy.columns = ['yy'+str(i) for i in range(col_len1)]
print(data_yy.head(2))

data_dy = data['导演'].str.split('/ ',expand=True)      # 导演数据分列
col_len2 = len(data_dy.columns)
data_dy.columns = ['dy'+str(i) for i in range(col_len2)]
print(data_dy.head(2))

data2 = data_yy.join(data_dy).join(data['name'])
print(data2.head(2))

    name    导演                                                 主演
0     团圆   王全安                          卢燕 / 凌峰 / 徐才根 / 马晓晴 / 莫小棋
1  非诚勿扰2   冯小刚   葛优 / 舒淇 / 孙红雷 / 姚晨 / 安以轩 / 邵兵 / 廖凡 / 邬逸聪 / 关晓...
    yy0  yy1   yy2   yy3   yy4   yy5   yy6   yy7   yy8   yy9  ...   yy55  \
0   卢燕   凌峰   徐才根   马晓晴    莫小棋  None  None  None  None  None  ...   None   
1   葛优   舒淇   孙红雷    姚晨   安以轩    邵兵    廖凡   邬逸聪   关晓彤   窦文涛   ...   None   

   yy56  yy57  yy58  yy59  yy60  yy61  yy62  yy63  yy64  
0  None  None  None  None  None  None  None  None  None  
1  None  None  None  None  None  None  None  None  None  

[2 rows x 65 columns]
    dy0   dy1   dy2   dy3   dy4   dy5   dy6   dy7   dy8   dy9  dy10  dy11
0   王全安  None  None  None  None  None  None  None  None  None  None  None
1   冯小刚  None  None  None  None  None  None  None  None  None  None  None
    yy0  yy1   yy2   yy3   yy4   yy5   yy6   yy7   yy8   yy9  ...     dy3  \
0   卢燕   凌峰   徐才根   马晓晴    莫小棋  None  None  None  None  None  ...    None   


In [18]:
# 数据拆分、合并

data_re = pd.DataFrame(columns=['name','导演','演员'])      # 创建一个空的Dataframe
col_yy = data_yy.columns                                    # 提取演员、导演的列名
col_dy = data_dy.columns

for dy in col_dy:
    for yy in col_yy: 
        data_i = data2[['name',dy,yy]].dropna()              # 提取数据
        data_i.columns = ['name','导演','演员']              # 列名重命名
        data_re = pd.concat([data_re,data_i])                # 添加数据
        
print(data_re.head())

       name    导演     演员
0        团圆   王全安    卢燕 
1     非诚勿扰2   冯小刚    葛优 
2     无蝉的夏天    苗月   吕聿来 
3  狄仁杰之通天帝国    徐克   刘德华 
4     我是植物人    王竞   李乃文 


In [20]:
# 汇总统计导演和演员的合作次数

res = data_re.groupby(['导演','演员']).count()               # 按照导演-演员进行计数统计，得到结果数据
res.reset_index(inplace=True)                                # reset_index() → 将所有索引级别转换为列
res.columns=['导演','演员','合作次数']
print(res.head())

writer = pd.ExcelWriter('result.xlsx')
res.to_excel(writer,'sheet1')
writer.save()
print('done')

       导演    演员  合作次数
0   (小)徐克   张欣      2
1   (小)徐克   黄莺      1
2   (小)徐克   刘彬      1
3   (小)徐克   叶露      1
4   (小)徐克   姜玉玲     2
done
